## 다중분류모델 - 강판 결함 예측

### 1. EDA & Feature Engineering

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
def load_dataset(csv_path):
    
    # 데이터셋 로드
    df = pd.read_csv(csv_path)
    
    # 인코딩 방식 변경
    # idxmax 함수는 각 행의 최대값을 가진 열의 인덱스를 반환한다. 따라서 원핫인코딩된 피쳐를 하나의 카테고리 변수로 복원할 수 있음
    df['Fault'] = df[['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']].idxmax(axis=1)
    
    # 라벨 인코딩
    encoder = LabelEncoder()
    df['Fault'] = encoder.fit_transform(df['Fault'])
    
    return df

In [2]:
# 특성공학
csv_path = '/mnt/c/Users/k10dh/AppData/Local/Packages/CanonicalGroupLimited.Ubuntu_79rhkp1fndgsc/k10dh/TeamProject/TeamProject1/KDH/Dataset/mulit_classification_data.csv'
df = load_dataset(csv_path)

# 이상치 제거
df = df[~((df['Pixels_Areas'] > 35000) |
          (df['X_Perimeter'] > 2000) |
          (df['Y_Perimeter'] > 2500) |
          (df['Sum_of_Luminosity'] > 0.5e7))]

# 'TypeOfSteel_A300'과 'TypeOfSteel_400'로 나누어진 특성을 하나로 통합
df['TypeOfSteel'] = df['TypeOfSteel_A300']
df.drop(['TypeOfSteel_A300', 'TypeOfSteel_A400'], axis=1, inplace=True)

# 다중공선성 문제 해결을 위해 상관관계가 높은 feature 확인 후 제거 (상관계수 절대값 0.9이상)
# 'X_Perimeter' -> 'Pixels_Areas'
# 'Y_Perimeter' -> 'X_Perimeter'
# 'Sum_of_Luminosity' -> 'Pixels_Areas', 'X_Perimeter'
# 'Pixels_Areas'는 특성중요도가 높으므로 활용하고 상관관계가 높은 나머지 특성들 제거
df.drop(['Sum_of_Luminosity', 'X_Perimeter', 'Y_Perimeter'], axis=1, inplace=True)

# 로그 스케일링(관측치 측정 범위가 넓음)
df['Log_Pixels_Areas'] = np.log(df['Pixels_Areas'])
df.drop(['Pixels_Areas'], axis=1, inplace=True)

# 중요도 낮은 특성 제거(머신러닝 분류모델에서 사용하는 기법인데 딥러닝에도 적용이 가능한가? -> 우선 모델 성능 개선은 되었음, 더 알아보면 좋을 것 같음)
df.drop(['Outside_Global_Index', 'SigmoidOfAreas', 'Log_Y_Index'], axis=1, inplace=True)

# 학습 데이터 분리
X = df.drop(['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults', 'Fault'], axis=1)
y = df['Fault']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state = 83)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1551, 20) (388, 20) (1551,) (388,)


In [3]:
df.columns

Index(['X_Minimum', 'X_Maximum', 'Y_Minimum', 'Y_Maximum',
       'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer',
       'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Square_Index',
       'Outside_X_Index', 'Edges_X_Index', 'Edges_Y_Index', 'LogOfAreas',
       'Log_X_Index', 'Orientation_Index', 'Luminosity_Index', 'Pastry',
       'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults',
       'Fault', 'TypeOfSteel', 'Log_Pixels_Areas'],
      dtype='object')

In [4]:
df.describe()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,Steel_Plate_Thickness,Edges_Index,Empty_Index,...,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults,Fault,TypeOfSteel,Log_Pixels_Areas
count,1939.000000,1939.000000,1.939000e+03,1.939000e+03,1939.000000,1939.000000,1939.000000,1939.000000,1939.000000,1939.000000,...,1939.000000,1939.000000,1939.000000,1939.000000,1939.000000,1939.000000,1939.00000,1939.000000,1939.000000,1939.000000
mean,571.707581,618.437855,1.649048e+06,1.649092e+06,84.626096,130.178442,1459.260959,78.777720,0.332032,0.413977,...,0.081485,0.097989,0.201135,0.037133,0.028365,0.207323,0.34657,2.569881,0.400722,5.733272
std,520.654729,497.665430,1.774493e+06,1.774491e+06,32.058785,18.691238,144.616025,55.100383,0.299704,0.136789,...,0.273650,0.297376,0.400952,0.189135,0.166057,0.405494,0.47600,1.763241,0.490171,1.808779
min,0.000000,4.000000,6.712000e+03,6.724000e+03,0.000000,37.000000,1227.000000,40.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.693147
25%,52.000000,192.500000,4.701220e+05,4.701520e+05,63.000000,124.000000,1358.000000,40.000000,0.060400,0.315750,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,2.000000,0.000000,4.430817
50%,436.000000,470.000000,1.199744e+06,1.199753e+06,90.000000,127.000000,1364.000000,70.000000,0.227800,0.412100,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,3.000000,0.000000,5.153292
75%,1053.000000,1072.500000,2.182309e+06,2.182322e+06,106.000000,140.000000,1650.000000,80.000000,0.575500,0.501400,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,3.000000,1.000000,6.695792
max,1705.000000,1713.000000,1.298766e+07,1.298769e+07,203.000000,253.000000,1794.000000,300.000000,0.995200,0.927500,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,6.000000,1.000000,10.145374


In [5]:
# 오버샘플링(학습데이터만)
from imblearn.over_sampling import SMOTE

# 오버샘플링 전 클래스 분포 확인
print('원본 데이터 크기 X_train: {}'.format(X_train.shape))
print('원본 데이터 크기 y_train: {} \n'.format(y_train.shape))

print("원본 데이터 '1' 개수: {}".format(sum(y_train==1)))
print("원본 데이터 '0' 개수: {} \n".format(sum(y_train==0)))

# SMOTE 적용
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

# 오버샘플링 후 클래스 분포 확인
print('샘플링 데이터 크기 X_train: {}'.format(X_train_res.shape))
print('샘플링 데이터 크기 y_train: {} \n'.format(y_train_res.shape))

print("샘플링 데이터 '1' 개수: {}".format(sum(y_train_res==1)))
print("샘플링 데이터 '0' 개수: {}".format(sum(y_train_res==0)))


원본 데이터 크기 X_train: (1551, 20)
원본 데이터 크기 y_train: (1551,) 

원본 데이터 '1' 개수: 43
원본 데이터 '0' 개수: 318 

샘플링 데이터 크기 X_train: (3710, 20)
샘플링 데이터 크기 y_train: (3710,) 

샘플링 데이터 '1' 개수: 530
샘플링 데이터 '0' 개수: 530


In [6]:
# MinMaxScaling
from sklearn.preprocessing import MinMaxScaler

# 스케일링할 피처 선택
scaling_features = X_train.columns

# 스케일링
scaler = MinMaxScaler()
X_train_scaled = X_train.copy()  # 원본 데이터 복사
X_test_scaled = X_test.copy()    # 원본 데이터 복사
X_train_scaled[scaling_features] = scaler.fit_transform(X_train[scaling_features])
X_test_scaled[scaling_features] = scaler.transform(X_test[scaling_features])

### 2. 모델 설계

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *

2023-05-24 01:55:14.289778: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
# 스킵 연결 모델(Skip Connection)
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Input, Dense, Dropout, Add
from tensorflow.keras.models import Model

def Classifier_Model_SC(units, l2):
    
    np.random.seed(42)
    tf.random.set_seed(42)
    
    # 입력층을 정의하며, 입력의 형태는 훈련 데이터의 특성 수에 따라 결정
    inputs = Input(shape=(len(X_train.keys()),))
    
    # 첫 번째 Dense 층을 만들고 ReLU 활성화 함수를 사용, L2 정규화 적용
    x = Dense(units=units[0], activation='relu', kernel_regularizer=regularizers.l2(l2[0]))(inputs)
    x = Dropout(0.2)(x)

    # 이후 7개의 Dense 층을 스킵 연결을 가지도록 생성
    for i in range(1, 8):   
        dense = Dense(units=units[i], activation='relu', kernel_regularizer=regularizers.l2(l2[i]))
        y = dense(x)
        y = Dropout(0.2)(y)
        
        # 현재 층의 출력(y)과 이전 층의 출력(x)을 더하여 스킵 연결 구현
        x = Add()([x, y])

    # 출력층을 정의, 유닛의 수는 클래스의 수와 동일하며, softmax 활성화 함수를 사용
    outputs = Dense(units=7, activation='softmax')(x)

    # 모델을 생성, 입력과 출력을 지정
    model = Model(inputs=inputs, outputs=outputs)
    
    # 옵티마이저와 손실 함수 설정
    optimizer = tf.keras.optimizers.Adam(
                                        learning_rate=0.001,
                                        beta_1=0.9,
                                        beta_2=0.999,
                                        epsilon=1e-08
                                        )

    model.compile(loss='sparse_categorical_crossentropy',  # 손실함수를 다중 클래스 분류에 적합한 형태로 변경
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    return model

In [ ]:
# 모델 객체 생성
units = [256 for _ in range(8)] # 노드 개수 설정
L2 = [0.001 + i * 0.0025 for i in range(8)] # L2 값 설정

model = Classifier_Model_SC(units, L2)

In [10]:
# 학습 자동 중단 설정
es = EarlyStopping(monitor='accuracy', patience=256, mode='auto')
rlrp = ReduceLROnPlateau(monitor='accuracy', factor=0.2, patience=256, mode='auto')

In [11]:
# 검증 정확도가 높은 모델의 가중치 저장
from tensorflow.keras.callbacks import Callback

# Callback 클래스를 상속받아 사용자 정의 콜백 클래스 생성
class CustomModelCheckpoint(Callback):
    def __init__(self, max_models_to_save=5):
        super().__init__()
        self.max_models_to_save = max_models_to_save # 최대로 저장할 모델의 수를 설정
        self.saved_models = [] # 저장된 모델들을 관리할 리스트를 생성
        
# 에포크 종료 시점에 호출되는 메서드를 정의
    def on_epoch_end(self, epoch, logs=None):
        val_accuracy = logs.get('val_accuracy', 0.0) # 현재 에포크의 검증 정확도
        model_weights = self.model.get_weights() # 현재 에포크의 모델 가중치

        # 첫 번째 에포크이거나 새로운 val_accuracy가 이전보다 높을 때만 모델 저장
        if not self.saved_models or val_accuracy > min(self.saved_models, key=lambda x: x[0])[0]:
            self.saved_models.append((val_accuracy, model_weights)) # (val_accuracy, model_weights) 형식의 튜플을 리스트에 추가
            # 리스트를 val_accuracy 기준으로 내림차순 정렬하고, 최대로 저장할 모델의 수를 넘지 않도록 리스트를 자름
            self.saved_models = sorted(self.saved_models, key=lambda x: -x[0])[:self.max_models_to_save] 

# 사용자 정의 콜백을 생성
custom_checkpoint = CustomModelCheckpoint(max_models_to_save=10)

In [12]:
# 하이퍼파라미터 세팅
LEARNING_RATE = 0.001
EPOCHS = 1024
MB_SIZE = 256
REPORT = 1
TRAIN_RATIO = 0.8

### 3. 모델 학습 및 평가

In [ ]:
# 모델 학습
np.random.seed(42)
tf.random.set_seed(42)

history = model.fit(
  X_train_scaled, y_train,
  batch_size=MB_SIZE,
  validation_split = 0.2,
  verbose=1,
  epochs=EPOCHS,
  callbacks=[es, rlrp, custom_checkpoint]
  )

In [ ]:
# test 정확도가 가장 높은 모델 찾기
# enumerate 함수를 사용하여 저장된 모델들의 인덱스와 검증 정확도, 가중치를 함께 호출
for i, (val_accuracy, weights) in enumerate(custom_checkpoint.saved_models):
    model.set_weights(weights) # 현재 순환에서의 모델 가중치를 로드
    test_metrics = model.evaluate(X_test_scaled, y_test, verbose=0) # 현재 가중치로 설정된 모델에 대해 테스트 데이터를 이용해 평가를 진행
    print(f'Model {i+1} - val_acc: {val_accuracy:.4f}, test_acc: {test_metrics[1]:.4f}') # test_metrics[1]은 evaluate 함수의 반환값 중 두 번째 값으로, 정확도를 의미

In [18]:
# test 정확도가 가장 높은 모델의 가중치를 로드(여기서는 Model 8이 제일 높았음)
model.set_weights(custom_checkpoint.saved_models[7][1])  # 0-based index이므로 8번째 모델은 7번 index

# 모델 가중치를 저장(git에 있는 'best_model_weights_classify.h5'는 가장 높은 성능 모델의 가중치이므로 중복 저장하지 않도록 주의)
# model.save_weights('best_model_weights_classify.h5')

In [19]:
# 테스트 세트 평가
model.load_weights("best_model_weights_classify.h5")

loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Loss:", round(loss, 3))
print("Test Accuracy:", round(accuracy, 3))

13/13 [==============================] - 0s 7ms/step - loss: 0.9645 - accuracy: 0.8067
Test Loss: 0.964
Test Accuracy: 0.807
